In [9]:
# Prepare data
%run testit.py

          Id  CDSEXE  MTREV  NBENF CDSITFAM       DTADH  CDTMT  CDDEM  \
0          1       2      0      0        A  1972-04-04      0      2   
1          2       2      0      0        A  1972-04-12      0      2   
2          3       2      0      0        B  1972-07-19      0      2   
3          4       3      0      0        M  1972-08-28      0      2   
4          5       2      0      0        A  1972-11-06      0      2   
...      ...     ...    ...    ...      ...         ...    ...    ...   
30327  30328       2      0      0        M  1982-06-16      0      2   
30328  30329       3      0      0        A  1980-02-29      0      2   
30329  30330       3      0      0        A  1980-09-17      0      2   
30330  30331       3      0      0        A  1980-07-03      0      2   
30331  30332       2      0      0        A  1983-07-29      0      2   

            DTDEM  ANNEE_DEM CDMOTDEM  CDCATCL  AGEAD rangagead  agedem  \
0      2003-06-25       2003       DV       21  

In [10]:
# Scoring
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Pre-proc
from sklearn import preprocessing

# Data Balance
"""
`pip install imbalanced-learn`
Ref: https://towardsdatascience.com/how-to-balance-a-dataset-in-python-36dff9d12704
"""
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced

# Plot
import matplotlib.pyplot as plt
import matplotlib

# Utils
from collections import Counter
import numpy as np
import pandas as pd

# Classifiers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [11]:
# Importer dans une dataframe pandas le CSV
prev_df = pd.read_csv('previsions.csv', delimiter=',')
# Afficher son contenu pour s'assurer qu'il est correctement lu
prev_df.head()

,dem,CDSEXE,MTREV,NBENF,AGEAD,CDTMT_0,CDTMT_2,CDTMT_4,CDTMT_6,CDSITFAM_A,...,CDCATCL_22,CDCATCL_23,CDCATCL_24,CDCATCL_25,CDCATCL_32,CDCATCL_40,CDCATCL_50,CDCATCL_61,CDCATCL_82,CDCATCL_98
0,1,2,0,0,33,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,55,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,0,39,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,29,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,0,0,45,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Conversion de la dataframe en numpy pour calculuer la moyenne et les X / Y
users_array = prev_df.to_numpy()
# Calcul de la "drop rate" moyenne, le taut de démission
average_drop_rate = (np.mean([i[0] for i in users_array]) * 100)

# Séparation des variables (X) de la cible de prévision (Y)
X = users_array[:, 1:]
Y = users_array[:, 0]

# Statistiques sur la transormation des données

In [13]:
# https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/

# Mise en place d'un compteur de distribution
counter = Counter(Y)

# Distribution graphique des résultats par deux facteurs discriminés par les valeurs de la cible

# Agead / MtRev
for label, _ in counter.items():
    row_ix = np.where(Y == label)[0]
    plt.scatter(X[row_ix, 3], X[row_ix, 1], label=str(label))
plt.legend()
plt.show()

# Agead / NbEnf
for label, _ in counter.items():
    row_ix = np.where(Y == label)[0]
    plt.scatter(X[row_ix, 3], X[row_ix, 2], label=str(label))
plt.legend()
plt.show()

# Décompte des valeurs de la cible
print(f"Distribution normale : {counter}")

<ipython-input-13-a13bfd130856>:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-13-a13bfd130856>:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Distribution normale : Counter({1: 30880, 0: 14474})


In [14]:
from sklearn.utils import compute_class_weight
# Calcul du poids des classes
class_weight = compute_class_weight('balanced', np.unique(Y), Y)

print(class_weight)

[1.56674036 0.73435881]


c:\users\hp\documents\github\bitume\venv\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


## Near Miss

Le principe du NearMiss est de réaliser un sous-échantillonage.
La référence se trouve à [l'adresse suivante](https://imbalanced-learn.org/stable/under_sampling.html)

Note: l'execution prends trop de temps pour mon PC

In [15]:
# Création d'un resampler NearMiss 1
nm1 = NearMiss(version=1)
# Équilibrage des échantillions
X_nearmiss, Y_nearmiss = nm1.fit_resample(X, Y)

# Mise en place d'un compteur de distribution
counter = Counter(Y_nearmiss)

# Décompte des valeurs de la cible
print(f"Sous-échantillonnage / distribution NearMiss : {counter}")

Sous-échantillonnage / distribution NearMiss : Counter({0: 14474, 1: 14474})


In [16]:
"""
# Création d'un resampler NearMiss 2
nm1 = NearMiss(version=2)
# Équilibrage des échantillions
X_nearmiss2, Y_nearmiss2 = nm1.fit_resample(X, Y)

# Mise en place d'un compteur de distribution
counter = Counter(Y_nearmiss2)

# Décompte des valeurs de la cible
print(f"Sous-échantillonnage / distribution NearMiss 2 : {counter}")
"""

'\n# Création d\'un resampler NearMiss 2\nnm1 = NearMiss(version=2)\n# Équilibrage des échantillions\nX_nearmiss2, Y_nearmiss2 = nm1.fit_resample(X, Y)\n\n# Mise en place d\'un compteur de distribution\ncounter = Counter(Y_nearmiss2)\n\n# Décompte des valeurs de la cible\nprint(f"Sous-échantillonnage / distribution NearMiss 2 : {counter}")\n'

## SMOTE
Le principe de la Technique de sur-échantillonnage des Minorités Synthétiques (SMOTE) est de réaliser un sur-échantillonage.

La référence se trouve à [l'adresse suivante](https://imbalanced-learn.org/stable/over_sampling.html)


Note: l'execution prends trop de temps pour mon PC

In [17]:
# Création d'un resampler et équilibrage des échantillions
X_smote, Y_smote = SMOTE().fit_resample(X, Y)

# Mise en place d'un compteur de distribution
counter = Counter(Y_smote)

# Décompte des valeurs de la cible
print(f"Sur-échantillonnage / distribution SMOTE : {counter}")

Sur-échantillonnage / distribution SMOTE : Counter({1: 30880, 0: 30880})


## ADASYN
Le principe de l'Adaptation Synthétique (ADASYN) est de réaliser un sur-échantillonage.

La référence se trouve à [l'adresse suivante](https://imbalanced-learn.org/stable/over_sampling.html)


In [18]:
# Création d'un resampler et équilibrage des échantillions
X_as, Y_as = ADASYN().fit_resample(X, Y)

# Mise en place d'un compteur de distribution
counter = Counter(Y_as)

# Décompte des valeurs de la cible
print(f"Sur-échantillonnage / distribution ADASYN : {counter}")



Sur-échantillonnage / distribution ADASYN : Counter({0: 31270, 1: 30880})


# Modèles

In [19]:
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

# Mise en place d'un modèle de régression linéaire, connu pour
# avoir un important rapport performance / temps d'execution
lr_model = LogisticRegression

# Mise en place d'un Support Vecteur Machine, présentant
# en général de meilleurs performances
svm_model = SVC()

# Mise en place d'un DummyClassifier permettant d'évaluer les performances
# des autres algoritmes
dummycl = DummyClassifier(strategy="most_frequent")

# Mise en place d'une classification naïve bayésienne
gmb = GaussianNB()

# Mise en place d'un arbre de décision
dectree = tree.DecisionTreeClassifier()

## Modèle 80/20 Régression Linéaire

In [20]:
# Découpage du corpus en apprentissage et test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(lr_model(random_state=42))
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_train, y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_test)

# Affichage du score
print(f'Score de la Régression Linéaire sans distribution {pipeline.score(X_test, y_test)}')

Score de la Régression Linéaire sans distribution 0.9212876198875537


c:\users\hp\documents\github\bitume\venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Les pipelines sont documentées à [l'adresse suivante](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

<br/>

### NearMiss sur modèle 80/20 Régression Linéaire

Le modèle actuel présente un ensemble déséquilibré (on possède plus de personnes ayant quitté qu'étant restées).
On utilise alors un ensemble de techniques, comme expliqué dans [cet article](https://heartbeat.fritz.ai/resampling-to-properly-handle-imbalanced-datasets-in-machine-learning-64d82c16ceaa)

Référence sur les pipelines déséquilibrées à [l'addresse suivante](https://imbalanced-learn.org/stable/under_sampling.html#controlled-under-sampling-techniques)


Note: Je n'arrive pas à faire tourner ces algo sur mon pc, ils demandent une trop grande puissance de calcul

In [21]:
# Création d'un resampler NearMiss
nm1 = NearMiss(version=1)

# Équilibrage des échantillions
X_resampled_nm1, Y_resampled = nm1.fit_resample(X, Y)

# Découpage du corpus en apprentissage et test
NM_X_train, NM_X_test, NM_y_train, NM_y_test = train_test_split(X_resampled_nm1, Y_resampled, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(lr_model(random_state=42))
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(NM_X_train, NM_y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(NM_X_test)

# Affichage du score
print(f'Précision de la Régression Linéaire avec Sous-échantillonnage NearMiss {pipeline.score(NM_X_test, NM_y_test)}')

Précision de la Régression Linéaire avec Sous-échantillonnage NearMiss 0.9381692573402418


c:\users\hp\documents\github\bitume\venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### SMOTE sur modèle 80/20 Régression Linéaire

Référence sur l'utilisation du Sur-échantillonage à [l'addresse suivante](https://imbalanced-learn.org/stable/over_sampling.html#from-random-over-sampling-to-smote-and-adasyn)

In [22]:
# Création et équilibrage des échantillions
X_smote, Y_smote = SMOTE().fit_resample(X, Y)

# Découpage du corpus en apprentissage et test
X_smote_train, X_smote_test, Y_smote_train, Y_smote_test = train_test_split(X_smote, Y_smote, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(lr_model(random_state=42))
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_smote_train, Y_smote_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_smote_test)

# Affichage du score
print(f'Précision de la Régression Linéaire avec Sur-échantillonnage SMOTE {pipeline.score(X_smote_test, Y_smote_test)}')

Précision de la Régression Linéaire avec Sur-échantillonnage SMOTE 0.9008257772020726


### ADASYN sur modèle 80/20 Régression Linéaire

Référence sur l'utilisation du Sur-échantillonage à [l'addresse suivante](https://imbalanced-learn.org/stable/over_sampling.html#from-random-over-sampling-to-smote-and-adasyn)

In [23]:
# Création et équilibrage des échantillions
X_as, Y_as = ADASYN().fit_resample(X, Y)

# Découpage du corpus en apprentissage et test
X_as_train, X_as_test, Y_as_train, Y_as_test = train_test_split(X_as, Y_as, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(lr_model(random_state=42))
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_as_train, Y_as_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_as_test)

# Affichage du score
print(f'Précision de la Régression Linéaire avec Sur-échantillonnage ADASYN {pipeline.score(X_as_test, Y_as_test)}')

Précision de la Régression Linéaire avec Sur-échantillonnage ADASYN 0.8363636363636363


### Indicateurs de classification

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)

print(f"""
Classification par défaut
Exactitude   : {accuracy}
Précision    : {precision}
Ratio Rappel : {recall}
F-score      : {f1}
""")


ValueError: Found input variables with inconsistent numbers of samples: [9071, 12430]

## Modèle 80/20 Classificateur factice

In [27]:
# Découpage du corpus en apprentissage et test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(lr_model(random_state=42))
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_train, y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_test)


# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(dummycl)
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_train, y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_test)

# Affichage du score
print(f'Score du Classificateur factice sans distribution {pipeline.score(X_test, y_test)}')

Score du Classificateur factice sans distribution 0.67853599382648


c:\users\hp\documents\github\bitume\venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Modèle 80/20 Gaussien

In [ ]:
# Découpage du corpus en apprentissage et test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(gmb)
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_train, y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_test)

# Affichage du score
print(f'Score du modèle Gaussien sans distribution {pipeline.score(X_test, y_test)}')



### Indicateurs de classification

In [ ]:
accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)

print(f"""
Classification par défaut
Exactitude   : {accuracy}
Précision    : {precision}
Ratio Rappel : {recall}
F-score      : {f1}
""")

## Modèle 80/20 Arbre de décision

In [ ]:
# Découpage du corpus en apprentissage et test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Création d'une pipeline avec un modèle linéaire
pipeline = make_pipeline(dectree)
# Entrainement avec les valeurs d'apprentissage
model = pipeline.fit(X_train, y_train)
# Calcul de la précision du modèle avec le corpus de test
prediction = model.predict(X_test)

# Affichage du score
print(f'Score de l\'Arbre de décision sans distribution {pipeline.score(X_test, y_test)}')



### Indicateurs de classification

In [ ]:
accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)

print(f"""
Classification par défaut
Exactitude   : {accuracy}
Précision    : {precision}
Ratio Rappel : {recall}
F-score      : {f1}
""")


## Modèle 80/20 Support Vector Machine

In [ ]:
# Découpage train / test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Mise en place de la pipeline SVM
pipeline = make_pipeline(svm_model)
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f'Score de SVM sans distribution {pipeline.score(X_test, y_test)}')

### NearMiss sur modèle 80/20 Support Vecteur Machine

In [ ]:
# NM_X_train, NM_X_test, NM_y_train, NM_y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
#
# NM_pipeline = make_pipeline_imb(NearMiss(random_state=42), lr_model(random_state=42))
# NM_model = NM_pipeline.fit(NM_X_train, NM_y_train)
# NM_prediction = NM_model.predict(NM_X_test)

# print(f'Précision de Support Vecteur Machine avec Sous-échantillonnage NearMiss {NM_pipeline.score(X_test, y_test)}')


### Indicateurs de classification

In [ ]:
accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)

print(f"""
Classification par défaut
Exactitude   : {accuracy}
Précision    : {precision}
Ratio Rappel : {recall}
F-score      : {f1}
""")

# Mise à l'échelle et Normalisation
## Régression Linéaire

In [ ]:
min_max = preprocessing.MinMaxScaler()
X_minmax = min_max.fit_transform(X)

std_scale = preprocessing.StandardScaler()
X_scale = std_scale.fit_transform(X)

X_l1 = preprocessing.normalize(X, norm="l1")
X_l2 = preprocessing.normalize(X, norm="l2")

### Calcul de la MAE, MSE, RMSE sur une Régression Linéaire

https://towardsdatascience.com/linear-regression-in-python-9a1f5f000606
https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error

In [ ]:
def get_lr_model(X_dat):
    X_train, X_test, y_train, y_test = train_test_split(X_dat, Y, test_size=0.2, random_state=42)
    pipeline = make_pipeline(lr_model())
    model = pipeline.fit(X_train, y_train)
    return X_test, y_test, model

X_test, y_test, model = get_lr_model(X)
prediction = model.predict(X_test)
print(f"MAE sur une Régression Linéaire sans normalisation {mean_absolute_error(y_test, prediction)}")
print(f"MSE sur une Régression Linéaire sans normalisation {mean_squared_error(y_test, prediction)}")
print(f"RMSE sur une Régression Linéaire sans normalisation {np.sqrt(mean_squared_error(y_test, prediction))}")
print("")

# X_test, y_test, model = get_lr_model(X_minmax)
# prediction = model.predict(X_test)
# print(f"MSE sur une Régression Linéaire avec MinMax {mean_squared_error(y_test, prediction)}")
#
X_test, y_test, model = get_lr_model(X_scale)
prediction = model.predict(X_test)
print(f"MAE sur une Régression Linéaire avec Répartitions Normales {mean_absolute_error(y_test, prediction)}")
print(f"MSE sur une Régression Linéaire avec Répartitions Normales {mean_squared_error(y_test, prediction)}")
print(f"RMSE sur une Régression Linéaire avec Répartitions Normales {np.sqrt(mean_squared_error(y_test, prediction))}")
print("")

X_test, y_test, model = get_lr_model(X_l1)
prediction = model.predict(X_test)
print(f"MAE sur une Régression Linéaire avec Normalisation L1 {mean_absolute_error(y_test, prediction)}")
print(f"MSE sur une Régression Linéaire avec Normalisation L1 {mean_squared_error(y_test, prediction)}")
print(f"RMSE sur une Régression Linéaire avec Normalisation L1 {np.sqrt(mean_squared_error(y_test, prediction))}")
print("")

X_test, y_test, model = get_lr_model(X_l2)
prediction = model.predict(X_test)
print(f"MAE sur une Régression Linéaire avec Normalisation L2 {mean_absolute_error(y_test, prediction)}")
print(f"MSE sur une Régression Linéaire avec Normalisation L2 {mean_squared_error(y_test, prediction)}")
print(f"RMSE sur une Régression Linéaire avec Normalisation L2 {np.sqrt(mean_squared_error(y_test, prediction))}")
print("")

### Exactitude après normaliastion et mise à l'échelle

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

pipeline = make_pipeline(lr_model(random_state=42))
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f'Exactitude de la Régression Linéaire sans distribution {pipeline.score(X_test, y_test)}')

# X_train, X_test, y_train, y_test = train_test_split(X_minmax, Y, test_size=0.2, random_state=42)
#
# pipeline = make_pipeline(lr_model(random_state=42))
# model = pipeline.fit(X_train, y_train)
# prediction = model.predict(X_test)
#
# print(f'Exactitude de la Régression Linéaire avec MinMax {pipeline.score(X_test, y_test)}')

X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.2, random_state=42)

pipeline = make_pipeline(lr_model(random_state=42))
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f'Exactitude de la Régression Linéaire avec Répartitions Normales {pipeline.score(X_test, y_test)}')

X_train, X_test, y_train, y_test = train_test_split(X_l1, Y, test_size=0.2, random_state=42)

pipeline = make_pipeline(lr_model(random_state=42))
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f'Exactitude de la Régression Linéaire avec Normalisation L1 {pipeline.score(X_test, y_test)}')

X_train, X_test, y_train, y_test = train_test_split(X_l2, Y, test_size=0.2, random_state=42)

pipeline = make_pipeline(lr_model(random_state=42))
model = pipeline.fit(X_train, y_train)
prediction = model.predict(X_test)

print(f'Exactitude de la Régression Linéaire avec Normalisation L2 {pipeline.score(X_test, y_test)}')



## Support Vecteur Machine

Note: j'ai pris la décision de ne *pas* réaliser ces tests en raison de mon toaster

In [ ]:
# TODO: code

# Validation Croisée

## Validation en 10-étapes (kFolds)
On fixe la valeur de k à k=10, qui est connue pour être la plus compêtante en général.
Source: https://machinelearningmastery.com/k-fold-cross-validation/

### Régression Linéaire

In [ ]:
from sklearn.model_selection import cross_val_score

def calc_accuracy_score(model, name, X, Y):
    scores = cross_val_score(model, X, Y, cv=10)
    scores_precision = cross_val_score(model, X, Y, cv=10, scoring='precision')
    scores_recall = cross_val_score(model, X, Y, cv=10, scoring='recall')
    scores_f1 = cross_val_score(model, X, Y, cv=10, scoring='f1')
    print(f"""
Évaluation de {name} sous Validation Croisée 10-étapes

Exactitude   : {scores.mean():.3f} (+/- {scores.std() * 2:.3f})
Précision    : {scores_precision.mean():.3f} (+/- {scores_precision.std() * 2:.3f})
Ratio Rappel : {scores_recall.mean():.3f} (+/- {scores_recall.std() * 2:.3f})
F-score      : {scores_f1.mean():.3f} (+/- {scores_f1.std() * 2:.3f})
    """)

In [ ]:
model = lr_model()
calc_accuracy_score(model, "la Régression Linéaire", X, Y)
# calc_accuracy_score(model, "la Régression Linéaire avec MinMax", X_minmax, Y)
# calc_accuracy_score(model, "la Régression Linéaire avec Scaler", X_scale, Y)
# calc_accuracy_score(model, "la Régression Linéaire avec Normalize 1", X_l1, Y)
# calc_accuracy_score(model, "la Régression Linéaire avec Normalize 2", X_l2, Y)

### Classificateur factice

In [ ]:
calc_accuracy_score(dummycl, "l'application du Classificateur factice", X, Y)

### Gaussien

In [ ]:
calc_accuracy_score(gmb, "la classification Gaussienne", X, Y)


### Arbre de décision

In [ ]:
calc_accuracy_score(dectree, "l'Arbre de décision", X, Y)

### Support Vecteur Machine

In [ ]:
# Je n'ai volontairement pas executé les calculs suivants afin de ne pas achever mon toaster

lsvm_model = SVC()
calc_accuracy_score(lsvm_model, "Support Vecteur Machine", X, Y)


# Modèles retenus

In [ ]:
nm1 = NearMiss(version=1)

# Équilibrage des échantillions
X_resampled_nm1, Y_resampled = nm1.fit_resample(X, Y)

model = LogisticRegression(random_state=42)

calc_accuracy_score(model, "la Régression Linéaire avec NearMiss", X_resampled_nm1, Y_resampled)

In [ ]:
nm1 = NearMiss(version=1)

std_scale = preprocessing.StandardScaler()
X_scale = std_scale.fit_transform(X)

# Équilibrage des échantillions
X_resampled_nm1, Y_resampled = nm1.fit_resample(X_scale, Y)

model = LogisticRegression(random_state=42)

calc_accuracy_score(model, "la Régression Linéaire avec NearMiss et StandardScaler", X_resampled_nm1, Y_resampled)